In [73]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anirudh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
data = pd.read_csv("twitter_training_data.csv", names=['target','id','date','flag','user','text'], encoding='ISO-8859-1')

In [75]:
remove_n = 1000000
drop_indices = np.random.choice(data.index, remove_n, replace=False)
data = data.drop(drop_indices)

In [76]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
17,0,1467813137,Mon Apr 06 22:20:25 PDT 2009,NO_QUERY,armotley,about to file taxes


In [77]:
#Check if the text column contains any null values
np.unique(data["text"].isnull())

array([False])

In [78]:
data.target = data.target.apply(lambda x: 1 if x == 4 else 0)

In [79]:
# Checking data split
np.unique(Counter(data["target"]))

array([Counter({1: 300093, 0: 299907})], dtype=object)

In [80]:
data = data.drop(['id','date','flag','user'], axis=1)

In [81]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [82]:
# Function to clean the dataset
def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

In [83]:
data.text = data.text.apply(lambda x: preprocess(x))

In [84]:
# Splitting the dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(data.text, data.target, test_size=0.3)

In [85]:
# Model Training

In [86]:
# Converting input to model format
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [87]:
final_x_train = tokenizer(x_train.values.tolist(), padding=True, truncation=True,max_length=80,return_tensors="pt")
final_x_test = tokenizer(x_test.values.tolist(), padding=True, truncation=True,max_length=80,return_tensors="pt")

Exception ignored in: <function tqdm.__del__ at 0x16b943760>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/tqdm/std.py", line 1162, in __del__
    self.close()
  File "/opt/homebrew/lib/python3.10/site-packages/tqdm/notebook.py", line 288, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x16b943760>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/tqdm/std.py", line 1162, in __del__
    self.close()
  File "/opt/homebrew/lib/python3.10/site-packages/tqdm/notebook.py", line 288, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x16b943760>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/tqdm/std.py", line 1162, in __del__
    sel

In [88]:
# train_input_ids = np.array(final_x_train["input_ids"])
# train_input_masks = np.array(final_x_train["token_type_ids"])
# train_segment_ids = np.array(final_x_train["attention_mask"])

In [89]:
# test_input_ids = np.array(final_x_test["input_ids"])
# test_input_masks = np.array(final_x_test["token_type_ids"])
# test_segment_ids = np.array(final_x_test["attention_mask"])

In [90]:
from torch.utils.data import TensorDataset
import torch
data_train = TensorDataset(final_x_train["input_ids"],final_x_train["attention_mask"], torch.tensor(y_train.values))
data_test = TensorDataset(final_x_test["input_ids"],final_x_test["attention_mask"],torch.tensor(y_test.values))

In [91]:
train_dl = DataLoader(data_train,sampler = RandomSampler(data_train),
                     batch_size = 32)
val_dl = DataLoader(data_test,sampler = SequentialSampler(data_test),
                     batch_size = 32)

In [92]:
len(train_dl),len(val_dl)

(13125, 5625)

In [93]:
from transformers import BertForSequenceClassification, AdamW
model = BertForSequenceClassification.from_pretrained(
'bert-base-uncased',
num_labels = 2,
output_attentions = False,
output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [94]:
import random
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [96]:
optimizer = AdamW(model.parameters(),lr = 2e-5,eps=1e-8)

/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [97]:
from transformers import get_linear_schedule_with_warmup
epochs = 1
total_steps = len(train_dl)*epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,
                                           num_training_steps=total_steps)

In [98]:
def accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat==label_flat)/len(label_flat)

In [99]:
def evaluate(dataloader_test):
    model.eval()
    loss_val_total = 0
    predictions,true_vals = [],[]
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    loss_val_avg = loss_val_total / len(dataloader_test)
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0)
    return loss_val_avg,predictions,true_vals

In [102]:
from tqdm.notebook import tqdm
from ipywidgets import FloatProgress
torch.cuda.empty_cache()
for epoch in range(1, epochs+1):
    
    model.train()
    
    loss_train_total = 0
    print('Epoch {:1d}'.format(epoch))
#     progress_bar = tqdm(train_dl, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in train_dl:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        print("training_loss: '{:.3f}'".format(loss.item()/len(batch)))
         
        
#     tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dl)            
    print(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dl)
    val_acc = accuracy(predictions, true_vals)
    print(f'Validation loss: {val_loss}')
    print(f'Accuracy: {val_acc}')

Epoch 1
training_loss: '0.251'
training_loss: '0.264'
training_loss: '0.240'
training_loss: '0.223'
training_loss: '0.240'
training_loss: '0.236'
training_loss: '0.239'
training_loss: '0.233'
training_loss: '0.239'
training_loss: '0.234'
training_loss: '0.235'
training_loss: '0.230'
training_loss: '0.229'
training_loss: '0.248'
training_loss: '0.235'
training_loss: '0.229'
training_loss: '0.230'
training_loss: '0.235'
training_loss: '0.232'
training_loss: '0.228'
training_loss: '0.233'
training_loss: '0.233'
training_loss: '0.229'
training_loss: '0.235'
training_loss: '0.238'
training_loss: '0.228'
training_loss: '0.232'
training_loss: '0.232'
training_loss: '0.236'
training_loss: '0.240'
training_loss: '0.225'
training_loss: '0.239'
training_loss: '0.232'
training_loss: '0.223'
training_loss: '0.245'
training_loss: '0.238'
training_loss: '0.231'
training_loss: '0.233'
training_loss: '0.231'
training_loss: '0.239'
training_loss: '0.227'
training_loss: '0.228'
training_loss: '0.231'
tra

KeyboardInterrupt: 

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(x_train)
print(f'Vectoriser fitted.')
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

Vectoriser fitted.
No. of feature_words:  500000


/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [23]:
X_train = vectoriser.transform(x_train)
X_test  = vectoriser.transform(x_test)
print(f'Data Transformed.')

Data Transformed.


In [32]:
def model_Evaluate(model):
    
    # Predict values for Test dataset
    y_pred = model.predict(X_test)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

    categories  = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 